# Data Prep

Loads the output of a [transfermark-webscrapper](https://github.com/dcaribou/transfermarkt-scraper) run, and a applies a series of transformations to produce a file that is validated and more friendly for perfoming analyisis. Some of these transformations are

* Creating handy ID columns
* Renaming fileds to comply with naming convention
* Parsing raw values into their own columns

## Load
Input to the data prep process is excepted to be the output of the [transfermark-webscrapper](https://github.com/dcaribou/transfermarkt-scraper). I. e., a file with JSON lines with one line per player containing all player appearances on a game up to the date the scraper was run.

Later on we will use [papermill](https://papermill.readthedocs.io/en/latest/usage-parameterize.html) to run the notebook from a dvc pipeline. The next cells define [notebook-wide parameters](https://papermill.readthedocs.io/en/latest/usage-parameterize.html) than can be overridded via papermill arguments.

In [1]:
raw_file = 'data/raw/appearances.json'
prep_file = 'data/prep/appearances.csv'

In [2]:
raw_file = f"../{raw_file}"
prep_file = f"../{prep_file}"

In [3]:
import pandas as pd

raw = pd.read_json(
  raw_file,
  lines=True,
  convert_dates=True,
  orient={'index','date'}
)

raw.head()

,confederation,domestic_competition,stats_competition,current_team,player_name,stats
0,europa,L1,AJ2,1-fc-koln,jens-castrop,"[{'matchday': '1', 'date': '2020-09-20', 'home..."
1,europa,L1,DFBJ,1-fc-koln,jens-castrop,"[{'matchday': 'First Round', 'date': '2020-10-..."
2,europa,L1,L1,1-fc-koln,kingsley-ehizibue,"[{'matchday': '1', 'date': '2020-09-19', 'home..."
3,europa,L1,DFB,1-fc-koln,kingsley-ehizibue,"[{'matchday': 'First Round', 'date': '2020-09-..."
4,europa,L1,L1,1-fc-koln,dominick-drexler,"[{'matchday': '1', 'date': '2020-09-19', 'home..."


## Prep
The prep phase applies a series of transformations on the raw data frame that we loaded above

In [4]:
from prep_lib import *

### Flatten
Firstly, we need to explode the data frame to have one row per player appearance, rather than one row per player

In [5]:
raw
raw_flat = flatten(raw, ['stats'])
raw_flat.head()

,matchday,date,home_team,away_team,result,pos,goals,assists,own_goals,yellow_cards,...,red_cards,substitutions_on,substitutions_off,minutes_played,transfermarkt_player_rating,confederation,domestic_competition,stats_competition,current_team,player_name
0,1,2020-09-20,fc-schalke-04-u19,1-fc-koln-u19,1:0,0,0,0,0,58',...,0,0,64',64,NaN,europa,L1,AJ2,1-fc-koln,jens-castrop
1,4,2020-10-23,1-fc-koln-u19,bayer-04-leverkusen-u19,5:0,0,0,0,0,65',...,0,0,74',74,NaN,europa,L1,AJ2,1-fc-koln,jens-castrop
2,First Round,2020-10-03,bayer-04-leverkusen-u19,1-fc-koln-u19,0:4,RW,0,0,0,0,...,0,0,0,90,NaN,europa,L1,DFBJ,1-fc-koln,jens-castrop
3,1,2020-09-19,1-fc-koln,tsg-1899-hoffenheim,2:3,RB,0,0,0,0,...,0,46',0,45,3.8,europa,L1,L1,1-fc-koln,kingsley-ehizibue
4,2,2020-09-26,arminia-bielefeld,1-fc-koln,1:0,RB,0,0,0,0,...,0,0,0,90,3.6,europa,L1,L1,1-fc-koln,kingsley-ehizibue


### Rename
Modify the names of the input columns to make them consistent with a naming convention

In [6]:
mappings = {
    'matchday': 'round',
    'home_team': 'home_club_name',
    'away_team': 'away_club_name',
    'current_team': 'player_club_name',
    'pos': 'player_position',
    'confederation': 'club_confederation',
    'domestic_competition': 'club_domestic_competition',
    'stats_competition': 'competition'
}
raw_flat
with_renamed_columns = renames(raw_flat, mappings)
with_renamed_columns.columns

Index(['round', 'date', 'home_club_name', 'away_club_name', 'result',
       'player_position', 'goals', 'assists', 'own_goals', 'yellow_cards',
       'second_yellow_cards', 'red_cards', 'substitutions_on',
       'substitutions_off', 'minutes_played', 'transfermarkt_player_rating',
       'club_confederation', 'club_domestic_competition', 'competition',
       'player_club_name', 'player_name'],
      dtype='object')

### Update
- [x] Convert `goals`, `assists`, `own_goals` and `date` to the appropriate types
- [x] Revamp `yellow_cards` and `red_cards`. `second_yellows` column is not needed
- [ ] Club name prettifying. _FC Watford_ instead of _fc-watford_
- [ ] Player name prettifying. _Adam Masina_ instead of _adam-masina_
- [x] Use longer names for `position` instead of the chryptic 'LB', etc. (use 'filter by position' [here](https://www.transfermarkt.co.uk/statistik/topscorer) to get the mappings)

In [7]:
with_improved_columns = improve_columns(with_renamed_columns)

### Create
- [x] Add surrogate keys `game_id`, `player_id`, `appearance_id`, `home_club_id`, `away_club_id`
- [x] Split `result` into `home_club_goals` and `away_club_goals`
- [x] Approximate appearance `season`

In [8]:

with_new_columns = add_new_columns(with_improved_columns)

### Filter
* Only season 2020 is complete on the current file, so we remove the rest
* To reduce the scope of this version of the data prep scritp, select only appearances from domestic competitions


In [9]:
with_filtered_appearances = filter_appearances(with_new_columns)

## Validate
Validate that the output dataframe contains consistent data. Two types of checks are performed.

### Value checks
- [x] Fields `red_cards`, `yellow_cards`, `own_goals`, `assists`, `goals` and `minutes_played` contain values within an expected range
- [x] Rows are unique on `player_id` + `date`
- [ ] `position` field is either one of the long form player positions from Transfermarkt

### Completeness checks
- [x] Number of teams per domestic competition must be exactly 20
- [ ] Each club must play 38 games per season on the domestic competition
- [ ] On each match, both clubs should have at least 11 appearances
- [ ] Similarly, each club must have at least 11 appearances per game


In [10]:
validations = [
    'assert_df_not_empty',
    'assert_minutes_played_gt_90',
    'assert_goals_in_range',
    'assert_assists_in_range',
    'assert_own_goals_in_range',
    'assert_yellow_cards_range',
    'assert_red_cards_range',
    'assert_unique_on_player_and_date',
    'assert_clubs_per_domestic_competition',
    # 'assert_games_per_season_per_club',
    'assert_appearances_per_match',
    # the next one catches wrongly tagged player appearance teams due to a transference in the middle if the season
    # uncomment after https://github.com/dcaribou/player-scores/issues/8 is resolved
    # 'assert_appearances_per_club_per_game',
    'assert_appearances_freshness_is_less_than_one_week',
    'assert_goals_ne_assists',
    'assert_goals_ne_own_goals',
    'assert_yellow_cards_not_constant',
    'assert_red_cards_not_constant'
]

failed_validations = validate(with_filtered_appearances, validations)
if failed_validations > 0:
    raise Exception(f"{failed_validations} validations did not pass")

## Save

In [11]:
with_filtered_appearances.to_csv(
  prep_file,
  index=False
)